## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

#os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

#TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
TRACKING_SERVER_HOST = "127.0.0.1"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='file:///c:/Users/aruna/Documents/MLOps_zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/1', creation_time=1764710908781, experiment_id='1', last_update_time=1764710908781, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/aruna/Documents/MLOps_zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1764710903236, experiment_id='0', last_update_time=1764710903236, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/12/02 16:50:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/02 16:50:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///c:/Users/aruna/Documents/MLOps_zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/1/55406b7c846d4a1a979fef9d06caed21/artifacts'
🏃 View run nimble-elk-678 at: http://127.0.0.1:5000/#/experiments/1/runs/55406b7c846d4a1a979fef9d06caed21
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/aruna/Documents/MLOps_zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/1', creation_time=1764710908781, experiment_id='1', last_update_time=1764710908781, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/aruna/Documents/MLOps_zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1764710903236, experiment_id='0', last_update_time=1764710903236, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1764711744013, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1764711744384, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1764711744384, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764711744384, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='7396f758dc0a489799eb9050fb490ffb', run_link='', source='models:/m-22711dbcf2a74c608887424822f4cd08', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

In [8]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2025/12/02 16:50:41 WARNING mlflow.tracking._model_registry.fluent: Run with id 55406b7c846d4a1a979fef9d06caed21 has no artifacts at artifact path 'models', registering model based on models:/m-f8b19ce7e63a409080016d9b3ec380fa instead
2025/12/02 16:50:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1764712241745, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764712241745, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='55406b7c846d4a1a979fef9d06caed21', run_link='', source='models:/m-f8b19ce7e63a409080016d9b3ec380fa', status='READY', status_message=None, tags={}, user_id='', version='2'>